In [1]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableMap
from IPython.display import Markdown, display
import logging
import os
import requests

In [2]:
# =========================
# General Config
# =========================
# Set up for bypassing system proxy
os.environ["no_proxy"] = "127.0.0.1,localhost"

# Set up logging for better error handling
DISABLE_LOGGING = False  # Set to True to disable logging, False to enable
logging.basicConfig(
    level=logging.CRITICAL if DISABLE_LOGGING else logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%H:%M:%S"
)
logger = logging.getLogger(__name__)

In [14]:
# ================================
# Configure Ollama host and port
# ================================
OLLAMA_HOST = os.getenv("OLLAMA_HOST", "127.0.0.1")
OLLAMA_PORT = os.getenv("OLLAMA_PORT", "11434")
OLLAMA_URL = f"http://{OLLAMA_HOST}:{OLLAMA_PORT}"

MODEL_NAME = "llama3.2:3b"
# MODEL_NAME = "gemma3:latest" 
# MODEL_NAME = "gpt-oss:latest"

In [15]:
# ===========================
# Define the prompt template
# ===========================
code_prompt = PromptTemplate(
    template="Write a very short {language} function that will {task}",
    input_variables=["language", "task"]
)
test_prompt = PromptTemplate(
    template="Write a test for the following {language} code:\n{code}",
    input_variables=["language", "code"]
)

# =========================
# Input parameters
# =========================
input_params = {"language": "python", "task": "return a list of 5 integers"}

In [16]:
# =========================
# Test server connection
# =========================
def test_server_connection(url: str) -> bool:
    try:
        response = requests.get(url, timeout=5)
        logger.info(f"Ollama server status: {response.status_code}")
        return True
    except requests.RequestException as e:
        logger.error(f"❌ Failed to connect to Ollama server at {url}: {e}")
        return False

In [ ]:
# # 1) Use Chatgpt api
# api_key = "sk-proj-Wxydnfibkzqrzu7Lw......."
# llm = OpenAI(
#     openai_api_key=api_key
# )

# 2) Use local LLM
# =================================================
# Function to invoke a model and display the result
# =================================================
result = dict()
def run_model(model_name, code_prompt: PromptTemplate, test_prompt: PromptTemplate, params: dict) -> None:
    try:
        logger.info(f"🚀 Running model: {model_name})")
        llm = OllamaLLM(model=model_name, base_url=OLLAMA_URL, timeout=30)


        # Step 1: Generate code
        code_chain = (
            RunnableMap({
                "task": lambda x: x["task"],
                "language": lambda x: x["language"]
            })
            | code_prompt
            | llm
            | RunnableLambda(lambda output: {
                "code": output.strip(),
                "language": input_params["language"],
                "task": input_params["task"]
            })
        )

        # Step 2: Generate test from code + language
        test_chain = (
            RunnableLambda(lambda x: {
                "code": x["code"],
                "language": x["language"]
            })
            | test_prompt
            | llm
        )

        # Step 3: Combine into sequence
        chain = RunnableSequence(
            code_chain 
            | RunnableLambda(lambda x: {
                "test": test_chain.invoke(x),
                "code": x['code']
                })
            )

        result = chain.invoke(input_params)

        print(">>>>>> GENERATED CODE:")
        print(result["code"])
        # display(Markdown(result["code"]))

        print(">>>>>> GENERATED TEST:")
        print(result["test"])
        # display(Markdown(result["test"]))
        
    except Exception as e:
        logger.error(f"⚠️ Error with {model_name}: {e}")

In [18]:
if not test_server_connection(OLLAMA_URL):
    exit(1)

# Run Model
run_model(MODEL_NAME, code_prompt, test_prompt, input_params)

09:06:23 - INFO - Ollama server status: 200
09:06:23 - INFO - 🚀 Running model: llama3.2:3b)
09:06:27 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
09:06:37 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


>>>>>> GENERATED CODE:
Here is a simple Python function to generate a list of 5 random integers:

```python
import random

def get_five_random_integers():
    """Returns a list of 5 random integers."""
    return [random.randint(0, 100) for _ in range(5)]
```

You can call this function like so:

```python
print(get_five_random_integers())
```

This will output something like `[23, 91, 19, 46, 63]`.
>>>>>> GENERATED TEST:
Here's an example of how you could write tests for the `get_five_random_integers` function using Python's built-in `unittest` module:

```python
import unittest
import random
from your_module import get_five_random_integers  # Replace 'your_module' with the actual name of the module where the function is defined

class TestGetFiveRandomIntegers(unittest.TestCase):

    def test_get_five_random_integers(self):
        """Test that get_five_random_integers returns a list of 5 integers."""
        result = get_five_random_integers()
        self.assertEqual(len(result), 

In [ ]:
print(result)


{'test': {'language': 'python', 'test': 'Here\'s an example test case using Python\'s built-in `unittest` module:\n```python\nimport unittest\nfrom your_module import generate_random_integers  # Replace \'your_module\' with the actual name of the file containing the function\n\nclass TestGenerateRandomIntegers(unittest.TestCase):\n    def test_generate_random_integers(self):\n        """Test that generate_random_integers returns a list of 5 integers."""\n        result = generate_random_integers()\n        self.assertEqual(len(result), 5)\n        for num in result:\n            self.assertIsInstance(num, int)\n\n    def test_generate_random_integers_in_range(self):\n        """Test that all generated numbers are within the range [0, 100]."""\n        result = generate_random_integers()\n        for num in result:\n            self.assertTrue(0 <= num <= 100)\n\n    def test_generate_random_integers_no_negative_numbers(self):\n        """Test that no negative numbers are generated."""\